In [1]:
import numpy as np
sgm = lambda x: 1/(1+np.exp(-x))
nrand = np.random.random
zerov = np.zeros

In [119]:
class RBM:
    '''A restricted Boltzmann machine model.'''
    def __init__(self, n_variables, n_hidden, spin = False):
        sigma = np.sqrt(4./(n_hidden+n_variables))
        self.a = sigma*(2*nrand(n_variables)-1)
        self.b = zerov(n_hidden)
        self.w = sigma*(2*nrand((n_variables,n_hidden))-1)
        self.min = -1*spin # Evaluates to zero if the model is not spin-based

    def activate(self, v, w, b, gap = 1):
        '''Turns the neurons of a forward layer on, according to a sigmoid probability function.'''
        vec = v@w + b # The neurons evaluation
        l = len(vec) # sample size
        prob = sgm(gap*vec)
        out = np.full(prob.shape,self.min)
        out[prob < nrand(prob.shape)] = 1
        return out
    
    def sample(self, v_data, steps = 1):
        h_data = self.activate(v_data, self.w, self.b)
        h_model = h_data
        for _ in range(steps):
            v_model = self.activate(h_model, self.w.T, self.a)
            h_model = self.activate(v_model, self.w, self.b)
        return h_data, v_model, h_model
    
    def grad(self, v_data, h_data, v_model, h_model):
        da = np.average(v_data - v_model, axis = 0)
        db = np.average(h_data - h_model, axis = 0)
        dw = sum(np.outer(v_data[i].T,h_data[i])-np.outer(v_model[i].T,h_model[i]) for i in range(len(v_data)))/len(v_data)
        #np.average(np.outer(v_data,h_data)-np.outer(v_model,h_model), axis = 1)
        return da, db, dw

    def train(self, data, n_epochs = 10, batch_size = 30, learning_rate = 1):
        batches = len(data)//batch_size
        n_variables = len(self.a)
        n_hidden = len(self.b)
        for epoch in range(n_epochs):
            np.random.shuffle(data)
            for i in range(batches):
                #v = data[i*batch_size:(i+1)*batch_size]
                #h, v_m, h_m = self.sample(v)
                #ca, cb, cw = self.grad(v,h,v_m,h_m)
                #self.a += learning_rate*ca
                #self.b += learning_rate*cb
                #self.w += learning_rate*cw
                for j, v in enumerate(data[i*batch_size:(i+1)*batch_size]):
                    if j%batch_size == 0:
                        v_data, v_model = np.zeros(n_variables), np.zeros(n_variables)
                        h_data, h_model = np.zeros(n_hidden), np.zeros(n_hidden)
                        vh_data, vh_model = np.zeros((n_variables, n_hidden)), np.zeros((n_variables, n_hidden))
                    h, v_m, h_m = self.sample(v)
                    v_data += v
                    h_data += h
                    v_model += v_m
                    h_model += h_m
                    vh_data += np.outer(v.T,h)
                    vh_model += np.outer(v_m.T,h_m)
                    if j%batch_size == batch_size - 1:
                        C = learning_rate / batch_size
                        dw = C*(vh_data - vh_model)
                        da = C*(v_data - v_model)
                        db = C*(h_data - h_model)
                        self.w += dw
                        self.a += da
                        self.b += db

            learning_rate = learning_rate/(0.05*learning_rate+1)



In [123]:
data = np.loadtxt('DATA/dataRBM_q0.1.csv', delimiter = ',')
rbm = RBM(len(data[0]),3)
rbm.train(data, n_epochs = 30, batch_size = 20)

<ipython-input-1-1887cda36eef>:2: RuntimeWarning: overflow encountered in exp
  sgm = lambda x: 1/(1+np.exp(-x))


In [131]:
np.random.shuffle(data)
_, v_fantasy, h_fantasy = rbm.sample(data,1)

<ipython-input-1-1887cda36eef>:2: RuntimeWarning: overflow encountered in exp
  sgm = lambda x: 1/(1+np.exp(-x))


In [132]:
v_fantasy

array([[0, 1, 0, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 1, 1],
       ...,
       [0, 1, 0, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 1, 1]])